In [ ]:
!pip install authlib
!pip install python-dotenv

In [ ]:
import sys
import time
import requests
import authlib
import os
import urllib.request
from dotenv import load_dotenv
from pathlib import Path
from typing import Optional, Dict, Union, Any
from authlib.jose import jwt
from requests import Response
import http.client
import json

In [ ]:
env_path = Path('.')/'tokens.env'

In [ ]:
load_dotenv(dotenv_path=env_path)

In [ ]:
API_KEY= os.getenv("USE_STEPS_ABOVE_TO_GET_YOUR_VERY_OWN_API_KEY")
API_SECRET= os.getenv("USE_STEPS_ABOVE_TO_GET_YOUR_VERY_OWN_API_SECRET")
ASSEMBLY_AI_TOKEN = os.getenv("SIGN_UP_AT_ASSEMBLYAI_TO_GET_YOUR_VERY_OWN_ASSEMBLYAI_API_TOKEN")
MEETING_ID = os.getenv('USE_STEPS_ABOVE_TO_FIND_YOUR_MEETING_ID')

In [ ]:
class Zoom:
    def __init__(
        self, api_key: str, api_secret: str
        # , userId: str, meetingId: str
        ):
        self.api_key = api_key
        self.api_secret = api_secret
        self.userId = 'me'
        self.meetingId = None
        self.base_url = "https://api.zoom.us/v2"
        self.reports_url = f"{self.base_url}/report/meetings"
        self.list_recordings_url = f"{self.base_url}/users/{self.userId}/recordings"
        self.get_recordings_url = f"{self.base_url}/meetings/{self.meetingId}/recordings"
        self.jwt_token_exp = 518400
        self.jwt_token_algo = "HS256"

    def generate_jwt_token(self) -> bytes:
        iat = int(time.time())

        jwt_payload: Dict[str, Any] = {
            "aud": None,
            "iss": self.api_key,
            "exp": iat + self.jwt_token_exp,
            "iat": iat
        }

        header: Dict[str, str] = {"alg": self.jwt_token_algo}

        jwt_token: bytes = jwt.encode(header, jwt_payload, self.api_secret)

        return jwt_token

In [ ]:
zoom = Zoom(API_KEY, API_SECRET)

In [ ]:
jwt_token: bytes = zoom.generate_jwt_token()
jwt_token_str = jwt_token.decode('UTF-8')
print(jwt_token_str)

In [ ]:
conn = http.client.HTTPSConnection("api.zoom.us")

headers = { 'authorization': 'Bearer ' + jwt_token_str}

conn.request(
    "GET", '/v2/meetings/' + MEETING_ID + '/recordings', headers=headers
    )

res = conn.getresponse()
data = res.read()

response_obj = (data.decode("utf-8"))

print(response_obj)

In [ ]:
meeting_dict = json.loads(response_obj)
download_url = meeting_dict['recording_files'][0]['download_url']
download_url

In [ ]:
authorized_url = download_url + "?access_token=" + jwt_token_str

In [ ]:
endpoint = 'https://api.assemblyai.com/v2/transcript'

json = {
    'audio_url': authorized_url
}

heads = {
    'authorization': ASSEMBLY_AI_TOKEN,
    'content-type': 'application/json'
}

resp = requests.post(endpoint, json=json, headers=heads)
print(resp.json())

In [ ]:
status_point = 'https://api.assemblyai.com/v2/transcript/' + resp.json()['id']

status_header = {'authorization':ASSEMBLY_AI_TOKEN} 

status_check = requests.get(status_point, headers=status_header)

print(status_check.json())

In [ ]:
status_check.json()['text']

# Transcribing Zoom recordings using the Zoom API and AssemblyAI in Python

![Zoom_90s](https://github.com/ddodds42/AssemblyAI_Data_Writing/raw/master/Zoom_API_Post/zoom90s.gif)

Wow, Zoom did itself a huge favor by rebranding since the 90's. This might actually be the main reason they survived the dotcom bubble.

Shame on you if that wry joke is over your head.

Zoom even has an API now folks! One of the many reasons to finally awaken from that 25 year coma! That's right, come back alive to the real simulation, and I'll show you how to use AssemblyAI to transcribe the words spoken in those Zoom meetings that have become such a ubiquitous aspect of our lives.

![matrix](https://github.com/ddodds42/AssemblyAI_Data_Writing/raw/master/Zoom_API_Post/matrix.gif)


## Creating a Cloud Recording on Zoom
The first thing to know when creating a cloud recording in Zoom is that you must have a Pro, Business, or Enterprise account. The basic free Zoom plan does not have a meeting recording feature enabled. You may already have such a plan, and may already have cloud recordings ready for transcription, in which case, you can skip to the next header and keep rolling.

But if you've just gotten an upgraded Zoom account for your project, and you still need to create a cloud recording, a few things to keep in mind:

- You do not need to have another person on the meeting for cloud recording to work. No need to schedule with someone else, just let it rip on your device and start the recording.

- The button looks like this if you're on a desktop, laptop, or large tablet:
![desktop_button](https://github.com/ddodds42/AssemblyAI_Data_Writing/raw/master/Zoom_API_Post/desktop_recording_button.jpg)

- If you're on a phone or small tablet, the cloud recording button hides down yonder behind those 3 dots.
![phone_button](https://github.com/ddodds42/AssemblyAI_Data_Writing/raw/master/Zoom_API_Post/phone_button.jpg)

- If you would like to test cloud recording and transcription with multiple people on the line, and you happen to be doing this with a friend's device that's in the same room, a couple of acoustic notes. You'll want to make sure that only one device as unmuted and producing sound, or the feedback will scratch at your eardrums.

- Another strange thing I noticed while cloud recording with multiple devices is that only the non-host's device will feed sound to the recording while you're muting one to prevent feedback. For some reason, if the host's device is unmuted while you're speaking, that sound will not be captured by Zoom's cloud recording.

- You might as well dodge both of those recording dilemmas by having each person and each device in separate rooms. That way the sound barriers prevent the feedback, and both parties can unmute.

Whew, now that those details are out of the way, end that meeting, and let's go find the cloud recording.

Upon ending the recording, a pop-up box will tell you that you will get an email once the recording is ready. Zoom's help pages cautiously warn users that it could take 24-48 hours for it to be ready, but that was likely the reality in early 2020 when the whole world flocked onto Zoom before their systems were ready for that level of traffic. It may still take a while for longer recordings, but I've found that here in 2021, for the short test recordings I've been transcribing, they are ready just about immediately.

Wow it feels great to talk trash about 2020 like it's a long-gone ex.

Anyway, to find your recording, sign into Zoom, and navigate to My Account.
![my_account](https://github.com/ddodds42/AssemblyAI_Data_Writing/raw/master/Zoom_API_Post/screencaps/mtg_my_account.JPG)

To the left, to the left, everything you want in a link to the left. Click on that "Recordings" tab over yonder on the left side.
![recordings](https://github.com/ddodds42/AssemblyAI_Data_Writing/raw/master/Zoom_API_Post/screencaps/mtg_recordings.JPG)

And now, the promised land! There you will see your cloud recordings, ready for you in mp4 and m4a format.
![recording_list](https://github.com/ddodds42/AssemblyAI_Data_Writing/raw/master/Zoom_API_Post/screencaps/mtg_recording_list.JPG)
Keep that open in a tab or write down the Meeting ID's of the meetings you want to transcribe, you'll need that 11-digit code once we touch code later.

## Accessing Zoom's API
Now, let's step through the process of getting keys to Zoom's backdoor. I'd like to give a shoutout to Billy Harmawan for [spelling out this convoluted process](https://medium.com/swlh/how-i-automate-my-church-organisations-zoom-meeting-attendance-reporting-with-python-419dfe7da58c). I'll give you the gist of it here too.

First, 